In [1]:
# Алексеев Д.П. (DSU-4,FEML-8)
# Домашнее задание к лекции «Функции потерь и оптимизация» (#3). Скорректированное

# Задание:
# Прочитать про методы оптимизации для нейронных сетей https://habr.com/post/318970/
# Реализовать самостоятельно логистическую регрессию
# Обучить ее методом градиентного спуска:
# -Методом nesterov momentum
# -Методом rmsprop

# Дополнительное задание *
# В качестве dataset’а взять Iris, оставив 2 класса:
# Iris Versicolor
# Iris Virginica

# Примечание: если я правильно понял задание, 
# то необходимо реализовать "вручную" логистическую регрессию для предсказаний классов 'Iris Versicolor' и 'Iris Virginica'

In [2]:
import pandas as pd
import numpy as np
from sklearn import datasets

In [3]:
# загружаем датасет с ирисами Фишера (сразу как датафрейм)
iris = datasets.load_iris(as_frame = True).frame
iris
# 4 входящих (описательных) признака | sepal - чашелистик, petal - лепесток
# значениe целевой переменной target (класс цветка): 'setosa' =0, 'versicolor'=1, 'virginica'=2

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,2
146,6.3,2.5,5.0,1.9,2
147,6.5,3.0,5.2,2.0,2
148,6.2,3.4,5.4,2.3,2


In [4]:
# по условию задачи, нам нужна классификация только 'versicolor' и 'virginica' ('versicolor'=1, 'virginica'=2)
# Следовательно, строки с классом=0, т.е. 'setosa', можно выкинуть.
iris = iris[iris['target'] != 0]
iris

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
50,7.0,3.2,4.7,1.4,1
51,6.4,3.2,4.5,1.5,1
52,6.9,3.1,4.9,1.5,1
53,5.5,2.3,4.0,1.3,1
54,6.5,2.8,4.6,1.5,1
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,2
146,6.3,2.5,5.0,1.9,2
147,6.5,3.0,5.2,2.0,2
148,6.2,3.4,5.4,2.3,2


In [5]:
X = iris[['sepal length (cm)', 'sepal width (cm)', 'petal length (cm)', 'petal width (cm)']]
X

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
50,7.0,3.2,4.7,1.4
51,6.4,3.2,4.5,1.5
52,6.9,3.1,4.9,1.5
53,5.5,2.3,4.0,1.3
54,6.5,2.8,4.6,1.5
...,...,...,...,...
145,6.7,3.0,5.2,2.3
146,6.3,2.5,5.0,1.9
147,6.5,3.0,5.2,2.0
148,6.2,3.4,5.4,2.3


In [6]:
# приведем целевую переменную к бинарному виду: 'versicolor'= 1-1 =0, 'virginica'= 2-1 = 1
Y = iris['target'] - 1
Y

50     0
51     0
52     0
53     0
54     0
      ..
145    1
146    1
147    1
148    1
149    1
Name: target, Length: 100, dtype: int32

In [9]:
# По теоретическим описаниям логистической регрессии возникли сложности с написанием работающего кода, 
# поэтому код частично пришлось позаимствовать по ссылке:
# https://pythobyte.com/logistic-regression-from-scratch-ae373d5d/

# Опишем класс логистического регрессора, определяющий функции, необходимые для обучения модели и предсказаний
class LogReg:
    def __init__(self,x,y):      
        self.intercept = np.ones((x.shape[0], 1))  
        self.x = np.concatenate((self.intercept, x), axis=1)
        self.weight = np.zeros(self.x.shape[1])
        self.y = y
        
    # Функция сигмоиды
    def sigmoid(self, x, weight):
        z = np.dot(x, weight)
        return 1 / (1 + np.exp(-z))
    
    # Функция потерь
    def loss(self, h, y):
        return (-y * np.log(h) - (1 - y) * np.log(1 - h)).mean()
    
    # Функция расчета градиента
    def gradient_descent(self, xx, h, y):
        return np.dot(xx.T, (h - y)) / y.shape[0]

    # Функция обучения регрессора "стандартным" способом
    def fit(self, lr , iterations):
        for i in range(iterations):
            # рассчитываем значение сигмоиды при текущих значениях весов признаков (w)
            sigma = self.sigmoid(self.x, self.weight)
            
            # рассчитываем значение функции потерь при текущем значении сигмоиды
            loss = self.loss(sigma, self.y)
            
            # рассчитываем шаг градиентного спуска
            dW = self.gradient_descent(self.x , sigma, self.y)
            
            # обновляем веса признаков с учетом заданного learning rate и рассчитанного шага градиентного спуска
            self.weight -= lr * dW
            print('Текущие веса признаков: ', self.weight)
            
        return print('Обучение модели закончено')
    
    # Функция обучения регрессора методом "Nesterov Momentum"
    def fit_nesterov(self, lr, iterations, imp):
        vx = np.zeros(self.x.shape[1])

        for i in range(iterations):
            sigma = self.sigmoid(self.x, self.weight)            
            
            dW = self.gradient_descent(self.x , sigma, self.y)
            
            #Обновляем накопленные вектора
            vx = imp * vx - lr * dW
            old_vx = vx
            
            #Обновляем веса
            self.weight = imp * old_vx + (1 - imp) * vx
            print('Текущие веса признаков: ', self.weight)
            
        return print('Обучение модели закончено')
    
    # Функция обучения регрессора методом "RMSProp"
    def fit_rmsprop(self, lr, iterations, imp, eps):
        eps_dW = np.zeros(self.x.shape[1])
        
        for i in range(iterations):
            sigma = self.sigmoid(self.x, self.weight)            
            
            dW = self.gradient_descent(self.x , sigma, self.y)
            
            #Обновляем накопленные вектора
            old_eps_dW = eps_dW
            eps_dW = imp*old_eps_dW + (1 - imp)*dW**2
            
            #Обновляем веса
            self.weight += -(lr * (1 / np.sqrt(eps_dW + eps)))*dW
            print('Текущие веса признаков: ', self.weight)
    
        return print('Обучение модели закончено')

    # Функция предсказания метки класса
    def predict(self, x_new , treshold):
        x_new = np.concatenate((self.intercept, x_new), axis=1)
        result = self.sigmoid(x_new, self.weight)
        result = result >= treshold
        y_pred = np.zeros(result.shape[0])
        for i in range(len(y_pred)):
            if result[i] == True: 
                y_pred[i] = 1
            else:
                continue
                
        return y_pred            

In [10]:
lr = 0.1
iterations = 100

# создадим регрессор, на трейн/тест выборки разбивать не будем
regressor = LogReg(X, Y)

# обучим регрессор
regressor.fit(lr, iterations)

Текущие веса признаков:  [0.     0.0163 0.0051 0.0323 0.0175]
Текущие веса признаков:  [-0.00755108 -0.01529732 -0.01169392  0.02670416  0.02196683]
Текущие веса признаков:  [-0.00832341 -0.00402241 -0.00887163  0.05482076  0.03796999]
Текущие веса признаков:  [-0.01511587 -0.03094459 -0.02350778  0.05269899  0.04356872]
Текущие веса признаков:  [-0.01649384 -0.02363706 -0.02247979  0.07746943  0.05836308]
Текущие веса признаков:  [-0.02267477 -0.04681455 -0.03538385  0.07808912  0.06484273]
Текущие веса признаков:  [-0.02453043 -0.04266201 -0.03577894  0.1001572   0.07864997]
Текущие веса признаков:  [-0.03021418 -0.06281771 -0.04728169  0.10294989  0.085816  ]
Текущие веса признаков:  [-0.0324472  -0.06118283 -0.04880871  0.12282127  0.09881041]
Текущие веса признаков:  [-0.03772425 -0.07888858 -0.05917187  0.12733796  0.10650934]
Текущие веса признаков:  [-0.04025481 -0.07926397 -0.06159907  0.14541572  0.11882999]
Текущие веса признаков:  [-0.0451977  -0.09497779 -0.07103232  0.151

In [11]:
# проверим качество предсказаний, возьмем порог 0.5
y_pred = regressor.predict(X, 0.5)
y_pred

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 1., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
       1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

In [12]:
print('Accuracy: ', sum(y_pred == Y) / Y.shape[0])

Accuracy:  0.96


Точность 0.96 - неплохой результат.
Попробуем теперь обучить модель методом Нестеров Моментум на тех же параметрах (lr = 0.1, iterations = 100).
Коэфф-т сохранения импульса возьмем равным 0.98.

In [13]:
lr = 0.1
iterations = 100
impulse = 0.98 

# создадим регрессор отдельно под Нестеров Моментум
regressor_NM = LogReg(X, Y)

# обучим регрессор методом Нестеров Моментум
regressor_NM.fit_nesterov(lr, iterations, impulse)

Текущие веса признаков:  [0.     0.0163 0.0051 0.0323 0.0175]
Текущие веса признаков:  [-0.00755108 -0.01562332 -0.01179592  0.02605816  0.02161683]
Текущие веса признаков:  [-0.00802018 -0.00307031 -0.00829634  0.05441769  0.03745046]
Текущие веса признаков:  [-0.01477648 -0.03071392 -0.02312521  0.05059664  0.04209242]
Текущие веса признаков:  [-0.01561159 -0.0212174  -0.02091579  0.0756104   0.05647925]
Текущие веса признаков:  [-0.02168319 -0.04524261 -0.03398527  0.07373582  0.06149837]
Текущие веса признаков:  [-0.02280324 -0.03825339 -0.03282013  0.09588463  0.07461815]
Текущие веса признаков:  [-0.02828053 -0.05919851 -0.04438143  0.09557719  0.07989842]
Текущие веса признаков:  [-0.02962046 -0.05427053 -0.04406156  0.11525814  0.09190145]
Текущие веса признаков:  [-0.03457908 -0.07258193 -0.05432361  0.11620901  0.0973504 ]
Текущие веса признаков:  [-0.03608589 -0.06934723 -0.05468551  0.13375528  0.10836436]
Текущие веса признаков:  [-0.04059    -0.08540011 -0.06382446  0.135

In [14]:
# проверим качество предсказаний, возьмем порог 0.5
y_pred = regressor_NM.predict(X, 0.5)
y_pred

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
       0., 1., 0., 1., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 1.,
       1., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

In [15]:
print('Accuracy: ', sum(y_pred == Y) / Y.shape[0])

Accuracy:  0.92


Мы видим, что точность предсказаний незначительно снизилась по сравнению с обычным способом (с 0.96 до 0.92), 
но качество модели всё еще осталось высоким.

Попробуем теперь оптимизацию методом RMSProp. Параметр epsilon для избежания деления на ноль возьмем равным 0.00001.

In [16]:
lr = 0.1
iterations = 100
impulse = 0.98 
epsilon = 0.00001

# создадим регрессор отдельно под RMSprop
regressor_RMSprop = LogReg(X, Y)

# обучим регрессор методом RMSprop
regressor_RMSprop.fit_rmsprop(lr, iterations, impulse, epsilon)

Текущие веса признаков:  [0.         0.70054575 0.64759621 0.70541843 0.70140421]
Текущие веса признаков:  [-0.70640061 -0.00549816 -0.05894919  0.00618602  0.01763495]
Текущие веса признаков:  [-0.44102053  0.30183348  0.23539464  0.3758326   0.44314053]
Текущие веса признаков:  [-0.92086932 -0.17044206 -0.23961615 -0.07913139  0.01205103]
Текущие веса признаков:  [-0.54139859  0.23486389  0.15772888  0.36193233  0.48303864]
Текущие веса признаков:  [-0.89197926 -0.10380488 -0.18497937  0.04529791  0.19168488]
Текущие веса признаков:  [-0.66603114  0.14263796  0.05516693  0.32075177  0.49237867]
Текущие веса признаков:  [-0.94883302 -0.12909022 -0.22038917  0.06944477  0.26389591]
Текущие веса признаков:  [-7.40154322e-01  9.75300591e-02  6.90764721e-04  3.21408994e-01
  5.37766990e-01]
Текущие веса признаков:  [-0.98111369 -0.13285038 -0.23335875  0.1102952   0.34776053]
Текущие веса признаков:  [-0.80910336  0.05517862 -0.05030185  0.32111207  0.5783341 ]
Текущие веса признаков:  [-

In [17]:
# проверим качество предсказаний, возьмем порог 0.5
y_pred = regressor_RMSprop.predict(X, 0.5)
y_pred

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1.,
       1., 0., 1., 1., 1., 0., 1., 0., 0., 0., 1., 0., 1., 0., 1., 0., 0.,
       1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

In [18]:
print('Accuracy: ', sum(y_pred == Y) / Y.shape[0])

Accuracy:  0.89


Точность предсказаний почему-то ещё снизилась (c 0.92 у Нестеров Моментум до 0.89 у RMSProp), причем ошибки в основном в предсказании класса virginica ("1" в бинарной классификации).

Самое высокое качество предсказаний получилось у модели обучения логистической регрессии стандартным способом (accuracy = 0.96).